In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [ ]:
print(torch.cuda.is_available())
print(torch.backends.cudnn.enabled)

In [ ]:
project_name = 'dogbreeds'
PATH = "data/dogscats/"
arch = resnext_101

In [ ]:
def get_data(bs = 64, sz = 224, val = True):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on,
                           max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test', num_workers=8,
               val_idxs=val_idxs if val else None, suffix='.jpg', tfms=tfms, bs=bs)

    return data if sz>300 else data.resize(340, 'tmp')

In [ ]:
learn = ConvLearner.pretrained(arch, 
                               data = get_data(bs = 64, sz = 250), 
                               precompute = True, 
                               ps = [0.5, 0.5, 0.5], 
                               xtra_fc=[500])

In [ ]:
lr_find = learn.lr_find()
learn.sched.plot()

In [ ]:
lr = 1e-4

In [ ]:
learn.fit(lr, 3)
learn.save(f"{project_name}_initial_fit")

In [1]:
learn.precompute = False
learn.fit(lr, 3)
learn.save(f"{project_name}_precompute")

NameError: name 'learn' is not defined

In [ ]:
lr_find = learn.lr_find()
learn.sched.plot()

In [ ]:
lr = 1e-3
lrs = [lr/6, lr/3, lr]

In [ ]:
learn.set_data(get_data(sz = 299, val = False))

In [ ]:
learn.unfreeze()
learn.bn_freeze(True) 
learn.fit(lrs, n_cycle = 2, cycle_len = 1, cycle_mult = 2)
learn.save(f"{project_name}_unfreeze")